In [2]:
from pymongo import MongoClient

In [3]:
client = MongoClient('')

In [4]:
db = client.get_database('production')

In [5]:
collection = db.get_collection('orders')

In [6]:
codes = db.get_collection('zipcodes')

In [7]:
import datetime as dt

In [8]:
now = dt.datetime.now()

In [9]:
current_month = now.month

In [10]:
records = collection.aggregate([
            {
                '$match': {
                    'address_id': {
                        '$exists': True
                    }, 
                    'createdOrderByShopify': None, 
                    'paid_status': 'PAGADO'
                }
            }, {
                '$lookup': {
                    'from': 'direcciones', 
                    'localField': 'address_id', 
                    'foreignField': '_id', 
                    'as': 'address'
                }
            }, {
                '$unwind': '$address'
            }, {
                '$match': {
                    'createdAt': {
                        '$gte': dt.datetime(2024, current_month, 1, 0, 0, 0, tzinfo=dt.timezone.utc), 
                        '$lt': dt.datetime.now()
                    }
                }
            }, {
                '$project': {
                    'cp': {
                        '$convert': {
                            'input': '$address.cp', 
                            'to': 'int', 
                            'onError': '', 
                            'onNull': ''
                        }
                    }, 
                    'kind_of_shop': 1
                }
            }, {
                '$group': {
                    '_id': {
                        'cp': '$cp', 
                        'kind_of_shop': '$kind_of_shop'
                    }, 
                    'sales': {
                        '$count': {}
                    }
                }
            },
            {
                '$project': {
                    'cp': '$_id.cp',
                    'kind_of_shop': '$_id.kind_of_shop',
                    'sales': 1,
                    '_id': 0
                }
            }
        ])

In [11]:
results = list(records)

In [12]:
import pandas as pd

In [54]:
from bson import ObjectId

In [13]:
df = pd.DataFrame(results)

In [14]:
df

,sales,cp,kind_of_shop
0,1,59513,BLIFE_APP_MX
1,1,44450,BLIFE_APP_MX
2,1,54600,MAYOREO
3,1,63039,BLIFE_APP_MX
4,1,80110,BLIFE_APP_MX
...,...,...,...
3593,1,60190,MAYOREO
3594,2,57150,BLIFE_APP_MX
3595,2,14240,BLIFE_APP_MX
3596,4,63195,BLIFE_APP_MX


In [16]:
df[df['sales'] > 1]

,sales,cp,kind_of_shop
2,19,45138,BLIFE_APP_MX
8,4,44200,BLIFE_APP_MX
9,3,64102,BLIFE_APP_MX
10,2,56334,BLIFE_APP_MX
13,4,64349,BLIFE_APP_MX
...,...,...,...
3514,3,56600,BLIFE_APP_MX
3515,2,99700,BLIFE_APP_MX
3520,3,21307,BLIFE_APP_MX
3523,2,54945,BLIFE_APP_MX


In [17]:
df['sales'].max()

np.int64(26)

In [15]:
zipcodes_records = codes.find()

In [16]:
df_zipcodes = pd.DataFrame(list(zipcodes_records))

In [17]:
df_zipcodes

,_id,d_codigo,D_mnpio,d_ciudad,d_estado,c_estado,d_asenta
0,66cf77f9dab1ab357d0e1b71,20000,Aguascalientes,Aguascalientes,Aguascalientes,1,[Aguascalientes Centro]
1,66cf77f9dab1ab357d0e1b72,20010,Aguascalientes,Aguascalientes,Aguascalientes,1,"[Colinas del Rio, Olivares Santana, Las Brisas..."
2,66cf77f9dab1ab357d0e1b73,20016,Aguascalientes,Aguascalientes,Aguascalientes,1,"[Colinas de San Ignacio, La Fundición, Fundici..."
3,66cf77f9dab1ab357d0e1b74,20018,Aguascalientes,Aguascalientes,Aguascalientes,1,[Línea de Fuego]
4,66cf77f9dab1ab357d0e1b75,20020,Aguascalientes,Aguascalientes,Aguascalientes,1,"[Buenos Aires, Circunvalación Norte, Las Arbol..."
...,...,...,...,...,...,...,...
32205,66cf77f9dab1ab357d0e993e,99990,Moyahua de Estrada,,Zacatecas,32,[Alameda Juárez]
32206,66cf77f9dab1ab357d0e993f,99993,Moyahua de Estrada,,Zacatecas,32,[Cuxpala]
32207,66cf77f9dab1ab357d0e9940,99994,Moyahua de Estrada,,Zacatecas,32,[Vicente Guerrero]
32208,66cf77f9dab1ab357d0e9941,99997,Moyahua de Estrada,,Zacatecas,32,[Las Palmas]


In [35]:
df_zipcodes[df_zipcodes['d_codigo'].eq(72120)]

,_id,d_codigo,D_mnpio,d_ciudad,d_estado,c_estado,d_asenta
21120,66cf77f9dab1ab357d0e6df1,72120,Puebla,Heroica Puebla de Zaragoza,Puebla,21,"[Los Arcos Santa Cruz, Ignacio Romero Vargas, ..."


In [1]:
def get_zipcode_id(row):
    res = df_zipcodes[df_zipcodes['d_codigo'].eq(row['cp'])]['_id'].tolist()
    if len(res) > 0:
        return res[0]
    print(row['cp'])
    return None

In [44]:
df_zipcodes[df_zipcodes['d_codigo'].eq(72120)]['_id']

21120    66cf77f9dab1ab357d0e6df1
Name: _id, dtype: object

In [45]:
str(df_zipcodes[df_zipcodes['d_codigo'].eq(72120)]['_id'])

'21120    66cf77f9dab1ab357d0e6df1\nName: _id, dtype: object'

In [46]:
df_zipcodes[df_zipcodes['d_codigo'].eq(72120)]['_id'].item()

ObjectId('66cf77f9dab1ab357d0e6df1')

In [49]:
str(df_zipcodes[df_zipcodes['d_codigo'].eq(72120)]['_id'].item())

'66cf77f9dab1ab357d0e6df1'

In [52]:
df_zipcodes[df_zipcodes['d_codigo'].eq(72120)]['_id'].tolist()

[ObjectId('66cf77f9dab1ab357d0e6df1')]

In [83]:
df_zipcodes[df_zipcodes['d_codigo'].eq(72120)]['_id'].tolist()[0]

ObjectId('66cf77f9dab1ab357d0e6df1')

In [63]:
df_zipcodes.loc[df_zipcodes['d_codigo'].eq(72120)]

,_id,d_codigo,D_mnpio,d_ciudad,d_estado,c_estado,d_asenta
21120,66cf77f9dab1ab357d0e6df1,72120,Puebla,Heroica Puebla de Zaragoza,Puebla,21,"[Los Arcos Santa Cruz, Ignacio Romero Vargas, ..."


In [78]:
df_zipcodes[df_zipcodes['d_codigo'].eq(72120)]['_id'].item()

ObjectId('66cf77f9dab1ab357d0e6df1')

In [18]:
df['zipcodes_id'] = df.apply(get_zipcode_id, axis=1)

5349
97248
64903
48282
54148
36251
62507

1537

7269
64833
50279
14038
80051
2450
4880
76760
22986
72216
45046
91725
62749
97294


In [94]:
df

,sales,cp,kind_of_shop,zipcodes_id
0,1,6720,BLIFE_ORDENES_MANUALES,66cf77f9dab1ab357d0e305b
1,1,64968,BLIFE_APP_MX,66cf77f9dab1ab357d0e6314
2,19,45138,BLIFE_APP_MX,66cf77f9dab1ab357d0e4746
3,1,58920,BLIFE_APP_MX,66cf77f9dab1ab357d0e58d4
4,1,57920,BLIFE_APP_MX,66cf77f9dab1ab357d0e573a
...,...,...,...,...
3522,1,70150,BLIFE_APP_MX,66cf77f9dab1ab357d0e6a3b
3523,2,54945,BLIFE_APP_MX,66cf77f9dab1ab357d0e543b
3524,2,1010,BLIFE_APP_MX,66cf77f9dab1ab357d0e2edb
3525,1,25198,BLIFE_APP_MX,66cf77f9dab1ab357d0e2151


In [20]:
def get_state(row):
    res = df_zipcodes[df_zipcodes['_id'].eq(row['zipcodes_id'])]['d_estado'].tolist()
    if len(res) > 0:
        return res[0]
    print(row['cp'])
    return None

In [21]:
df['state'] = df.apply(get_state, axis=1)

5349
97248
64903
48282
54148
36251
62507

1537

7269
64833
50279
14038
80051
2450
4880
76760
22986
72216
45046
91725
62749
97294


In [23]:
df

,sales,cp,kind_of_shop,zipcodes_id,state
0,1,59513,BLIFE_APP_MX,66cf77f9dab1ab357d0e59c0,Michoacán de Ocampo
1,1,44450,BLIFE_APP_MX,66cf77f9dab1ab357d0e4698,Jalisco
2,1,54600,MAYOREO,66cf77f9dab1ab357d0e53c7,México
3,1,63039,BLIFE_APP_MX,66cf77f9dab1ab357d0e6099,Nayarit
4,1,80110,BLIFE_APP_MX,66cf77f9dab1ab357d0e7b2e,Sinaloa
...,...,...,...,...,...
3593,1,60190,MAYOREO,66cf77f9dab1ab357d0e5ae0,Michoacán de Ocampo
3594,2,57150,BLIFE_APP_MX,66cf77f9dab1ab357d0e5706,México
3595,2,14240,BLIFE_APP_MX,66cf77f9dab1ab357d0e324e,Ciudad de México
3596,4,63195,BLIFE_APP_MX,66cf77f9dab1ab357d0e60dd,Nayarit


In [22]:
def get_state_name(row):
    res = df_zipcodes[df_zipcodes['d_codigo'].eq(row['cp'])]['d_estado'].tolist()
    if len(res) > 0:
        return res[0]
    print(row['cp'])
    return None

In [24]:
df['state_name'] = df.apply(get_state_name, axis=1)

5349
97248
64903
48282
54148
36251
62507

1537

7269
64833
50279
14038
80051
2450
4880
76760
22986
72216
45046
91725
62749
97294


In [25]:
df

,sales,cp,kind_of_shop,zipcodes_id,state,state_name
0,1,59513,BLIFE_APP_MX,66cf77f9dab1ab357d0e59c0,Michoacán de Ocampo,Michoacán de Ocampo
1,1,44450,BLIFE_APP_MX,66cf77f9dab1ab357d0e4698,Jalisco,Jalisco
2,1,54600,MAYOREO,66cf77f9dab1ab357d0e53c7,México,México
3,1,63039,BLIFE_APP_MX,66cf77f9dab1ab357d0e6099,Nayarit,Nayarit
4,1,80110,BLIFE_APP_MX,66cf77f9dab1ab357d0e7b2e,Sinaloa,Sinaloa
...,...,...,...,...,...,...
3593,1,60190,MAYOREO,66cf77f9dab1ab357d0e5ae0,Michoacán de Ocampo,Michoacán de Ocampo
3594,2,57150,BLIFE_APP_MX,66cf77f9dab1ab357d0e5706,México,México
3595,2,14240,BLIFE_APP_MX,66cf77f9dab1ab357d0e324e,Ciudad de México,Ciudad de México
3596,4,63195,BLIFE_APP_MX,66cf77f9dab1ab357d0e60dd,Nayarit,Nayarit


In [26]:
grouped_by_state = df.groupby(df['state'])

In [27]:
grouped_by_state.count()

,sales,cp,kind_of_shop,zipcodes_id,state_name
state,,,,,
Aguascalientes,63,63,63,63,63
Baja California,94,94,94,94,94
Baja California Sur,35,35,35,35,35
Campeche,17,17,17,17,17
Chiapas,56,56,56,56,56
Chihuahua,112,112,112,112,112
Ciudad de México,493,493,493,493,493
Coahuila de Zaragoza,81,81,81,81,81
Colima,32,32,32,32,32


In [28]:
grouped_by_state_kind_of_shop = df.groupby([df['state'], df['kind_of_shop']])

In [29]:
grouped_by_state_kind_of_shop.value_counts()

state           kind_of_shop            sales  cp     zipcodes_id               state_name    
Aguascalientes  BLIFE_APP_MX            1      20020  66cf77f9dab1ab357d0e1b75  Aguascalientes    1
                                               20030  66cf77f9dab1ab357d0e1b77  Aguascalientes    1
                                               20040  66cf77f9dab1ab357d0e1b78  Aguascalientes    1
                                               20050  66cf77f9dab1ab357d0e1b7a  Aguascalientes    1
                                               20059  66cf77f9dab1ab357d0e1b7b  Aguascalientes    1
                                                                                                 ..
Zacatecas       BLIFE_APP_MX            4      98612  66cf77f9dab1ab357d0e972f  Zacatecas         1
                BLIFE_ORDENES_MANUALES  1      98050  66cf77f9dab1ab357d0e9663  Zacatecas         1
                MAYOREO                 1      98090  66cf77f9dab1ab357d0e967a  Zacatecas         1
     

In [30]:
grouped_by_state_kind_of_shop.size()

state            kind_of_shop          
Aguascalientes   BLIFE_APP_MX              55
                 BLIFE_ORDENES_MANUALES     2
                 MAYOREO                    6
Baja California  BLIFE_APP_MX              83
                 BLIFE_ORDENES_MANUALES     4
                                           ..
Yucatán          BLIFE_APP_MX              29
                 MAYOREO                    4
Zacatecas        BLIFE_APP_MX              42
                 BLIFE_ORDENES_MANUALES     1
                 MAYOREO                    3
Length: 85, dtype: int64

In [31]:
grouped_by_state_kind_of_shop.size().to_frame().to_records()

rec.array([('Aguascalientes', 'BLIFE_APP_MX',  55),
           ('Aguascalientes', 'BLIFE_ORDENES_MANUALES',   2),
           ('Aguascalientes', 'MAYOREO',   6),
           ('Baja California', 'BLIFE_APP_MX',  83),
           ('Baja California', 'BLIFE_ORDENES_MANUALES',   4),
           ('Baja California', 'MAYOREO',   7),
           ('Baja California Sur', 'BLIFE_APP_MX',  32),
           ('Baja California Sur', 'BLIFE_ORDENES_MANUALES',   1),
           ('Baja California Sur', 'MAYOREO',   2),
           ('Campeche', 'BLIFE_APP_MX',  17),
           ('Chiapas', 'BLIFE_APP_MX',  48),
           ('Chiapas', 'BLIFE_ORDENES_MANUALES',   2),
           ('Chiapas', 'MAYOREO',   6),
           ('Chihuahua', 'BLIFE_APP_MX', 100),
           ('Chihuahua', 'BLIFE_ORDENES_MANUALES',   2),
           ('Chihuahua', 'MAYOREO',  10),
           ('Ciudad de México', 'BLIFE_APP_MX', 453),
           ('Ciudad de México', 'BLIFE_ORDENES_MANUALES',  26),
           ('Ciudad de México', 'MAYOREO',  14),


In [32]:
df['state'].value_counts()

state
Ciudad de México                   493
México                             449
Jalisco                            296
Nuevo León                         192
Puebla                             189
Guanajuato                         173
Veracruz de Ignacio de la Llave    161
Michoacán de Ocampo                141
Chihuahua                          112
Sonora                             105
Sinaloa                            105
Baja California                     94
Tamaulipas                          87
Querétaro                           86
Coahuila de Zaragoza                81
Morelos                             72
San Luis Potosí                     68
Oaxaca                              66
Aguascalientes                      63
Hidalgo                             59
Guerrero                            57
Chiapas                             56
Tabasco                             51
Quintana Roo                        51
Zacatecas                           46
Nayarit            

In [33]:
df['state'].value_counts().to_frame().to_records()

rec.array([('Ciudad de México', 493), ('México', 449), ('Jalisco', 296),
           ('Nuevo León', 192), ('Puebla', 189), ('Guanajuato', 173),
           ('Veracruz de Ignacio de la Llave', 161),
           ('Michoacán de Ocampo', 141), ('Chihuahua', 112),
           ('Sonora', 105), ('Sinaloa', 105), ('Baja California',  94),
           ('Tamaulipas',  87), ('Querétaro',  86),
           ('Coahuila de Zaragoza',  81), ('Morelos',  72),
           ('San Luis Potosí',  68), ('Oaxaca',  66),
           ('Aguascalientes',  63), ('Hidalgo',  59), ('Guerrero',  57),
           ('Chiapas',  56), ('Tabasco',  51), ('Quintana Roo',  51),
           ('Zacatecas',  46), ('Nayarit',  45),
           ('Baja California Sur',  35), ('Durango',  35),
           ('Yucatán',  33), ('Colima',  32), ('Tlaxcala',  24),
           ('Campeche',  17)],
          dtype=[('state', 'O'), ('count', '<i8')])

In [34]:
record_results = df['state'].value_counts().to_frame().to_records()

In [35]:
list(record_results)

[np.record(('Ciudad de México', 493), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('México', 449), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Jalisco', 296), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Nuevo León', 192), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Puebla', 189), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Guanajuato', 173), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Veracruz de Ignacio de la Llave', 161), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Michoacán de Ocampo', 141), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Chihuahua', 112), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Sonora', 105), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Sinaloa', 105), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Baja California', 94), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Tamaulipas', 87), dtype=[('state', 'O'), ('count', '<i8')]),
 np.r

In [36]:
record_results = grouped_by_state_kind_of_shop.size().to_frame()

In [37]:
record_results

0
state           kind_of_shop              
Aguascalientes  BLIFE_APP_MX            55
                BLIFE_ORDENES_MANUALES   2
                MAYOREO                  6
Baja California BLIFE_APP_MX            83
                BLIFE_ORDENES_MANUALES   4
...                                     ..
Yucatán         BLIFE_APP_MX            29
                MAYOREO                  4
Zacatecas       BLIFE_APP_MX            42
                BLIFE_ORDENES_MANUALES   1
                MAYOREO                  3

[85 rows x 1 columns]

In [38]:
dict_result = record_results.to_dict('dict')

In [39]:
dict_result

{0: {('Aguascalientes', 'BLIFE_APP_MX'): 55,
  ('Aguascalientes', 'BLIFE_ORDENES_MANUALES'): 2,
  ('Aguascalientes', 'MAYOREO'): 6,
  ('Baja California', 'BLIFE_APP_MX'): 83,
  ('Baja California', 'BLIFE_ORDENES_MANUALES'): 4,
  ('Baja California', 'MAYOREO'): 7,
  ('Baja California Sur', 'BLIFE_APP_MX'): 32,
  ('Baja California Sur', 'BLIFE_ORDENES_MANUALES'): 1,
  ('Baja California Sur', 'MAYOREO'): 2,
  ('Campeche', 'BLIFE_APP_MX'): 17,
  ('Chiapas', 'BLIFE_APP_MX'): 48,
  ('Chiapas', 'BLIFE_ORDENES_MANUALES'): 2,
  ('Chiapas', 'MAYOREO'): 6,
  ('Chihuahua', 'BLIFE_APP_MX'): 100,
  ('Chihuahua', 'BLIFE_ORDENES_MANUALES'): 2,
  ('Chihuahua', 'MAYOREO'): 10,
  ('Ciudad de México', 'BLIFE_APP_MX'): 453,
  ('Ciudad de México', 'BLIFE_ORDENES_MANUALES'): 26,
  ('Ciudad de México', 'MAYOREO'): 14,
  ('Coahuila de Zaragoza', 'BLIFE_APP_MX'): 78,
  ('Coahuila de Zaragoza', 'MAYOREO'): 3,
  ('Colima', 'BLIFE_APP_MX'): 29,
  ('Colima', 'MAYOREO'): 3,
  ('Durango', 'BLIFE_APP_MX'): 34,
  ('Dur

In [40]:
count_total = grouped_by_state_kind_of_shop['state'].count().reset_index(name='total')

In [41]:
count_total

,state,kind_of_shop,total
0,Aguascalientes,BLIFE_APP_MX,55
1,Aguascalientes,BLIFE_ORDENES_MANUALES,2
2,Aguascalientes,MAYOREO,6
3,Baja California,BLIFE_APP_MX,83
4,Baja California,BLIFE_ORDENES_MANUALES,4
...,...,...,...
80,Yucatán,BLIFE_APP_MX,29
81,Yucatán,MAYOREO,4
82,Zacatecas,BLIFE_APP_MX,42
83,Zacatecas,BLIFE_ORDENES_MANUALES,1


In [50]:
count_total.to_dict('records')

[{'state': 'Aguascalientes', 'kind_of_shop': 'BLIFE_APP_MX', 'total': 55},
 {'state': 'Aguascalientes',
  'kind_of_shop': 'BLIFE_ORDENES_MANUALES',
  'total': 2},
 {'state': 'Aguascalientes', 'kind_of_shop': 'MAYOREO', 'total': 6},
 {'state': 'Baja California', 'kind_of_shop': 'BLIFE_APP_MX', 'total': 83},
 {'state': 'Baja California',
  'kind_of_shop': 'BLIFE_ORDENES_MANUALES',
  'total': 4},
 {'state': 'Baja California', 'kind_of_shop': 'MAYOREO', 'total': 7},
 {'state': 'Baja California Sur', 'kind_of_shop': 'BLIFE_APP_MX', 'total': 32},
 {'state': 'Baja California Sur',
  'kind_of_shop': 'BLIFE_ORDENES_MANUALES',
  'total': 1},
 {'state': 'Baja California Sur', 'kind_of_shop': 'MAYOREO', 'total': 2},
 {'state': 'Campeche', 'kind_of_shop': 'BLIFE_APP_MX', 'total': 17},
 {'state': 'Chiapas', 'kind_of_shop': 'BLIFE_APP_MX', 'total': 48},
 {'state': 'Chiapas', 'kind_of_shop': 'BLIFE_ORDENES_MANUALES', 'total': 2},
 {'state': 'Chiapas', 'kind_of_shop': 'MAYOREO', 'total': 6},
 {'state':